Mitq tutorial: https://mitiq.readthedocs.io/en/stable/examples/ibmq-backends.html

# Imports

In [1]:
import numpy as np

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise

from utils.pce_vs_zne_utils import *

from qiskit import *

from utils.pauli_checks import convert_to_PCS_circ # new util

from mitiq import zne

from utils.pce_vs_zne_utils import *

from qiskit_ibm_runtime.fake_provider import *
from qiskit_aer import AerSimulator

np.set_printoptions(precision=6, edgeitems=10, linewidth=150, suppress=True)

#### Backend settings

In [2]:
USE_REAL_HARDWARE = False

In [3]:
# Fake backend noise model
# fake_backend = FakeMelbourneV2()
# noise_model = noise.NoiseModel.from_backend(fake_backend)

In [4]:
# Custom nosie model
prob_1 = 1e-3 # 1-qubit gate
prob_2 = 1e-2  # 2-qubit gate

error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'sx', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

# test set
# noise_model.add_all_qubit_quantum_error(error_1, ['z', 'y', 'x', 's', 'sdg', 'h'])
# noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

In [ ]:
service = QiskitRuntimeService()
print(service.instances())

In [ ]:
if QiskitRuntimeService.saved_accounts() and USE_REAL_HARDWARE:
    service = QiskitRuntimeService()
    backend = service.least_busy(operational=True, simulator=False)
    noise_model = False
else:
    backend = AerSimulator(noise_model=noise_model, method="stabilizer")

print(backend)

# Run Experiments 

In [ ]:
# 1) Your sweep parameters
qubit_list  = [6, 8, 10]
depth_list  = [10, 20, 30, 40, 50, 60]
all_results = {}

for num_qubits in qubit_list:
    for circuit_depth in depth_list:
        print(f"\n=== Running for {num_qubits=} qubits, {circuit_depth=} depth ===")

        # --- STEP 1: experiment setup ---
        pauli_string      = "Z" * num_qubits
        num_circs         = 20
        num_checks_to_fit = num_qubits // 2
        only_Z_checks = True
        shots_per_check   = 10_000

        # --- STEP 2: generate your random Clifford circuits ---
        cliff_circs = random_cliff_circs(
            num_qubits, circuit_depth, num_circs, pauli_string
        )

        # --- STEP 3: compute ideal expectations ---
        ideal_expectations = [
            get_ideal_expectation(circ, pauli_string)
            for circ in cliff_circs
        ]

        # --- STEP 4: precompute all the PCS circuits & signs ---
        # (so you can index them by [i][j] below)
        pcs_circs   = []  # will be a list of length num_circs, each entry is a list of pcs circuits for that cliff circuit
        signs_list  = []  # same shape, but holds the ±1 sign for each check

        for circ in cliff_circs:
            circ_pcs  = []
            circ_signs = []
            # assume num_checks_to_fit is the max number of checks you want
            for check_id in range(1, num_checks_to_fit + 1):
                sign, pcs_circ = convert_to_PCS_circ(circ, num_qubits, check_id, only_Z_checks=only_Z_checks, barriers=True)
                circ_pcs.append(pcs_circ)
                circ_signs.append(sign)
            pcs_circs.append(circ_pcs)
            signs_list.append(circ_signs)

        # --- UPDATED ZNE TESTS: sweeping over methods, scale factor sets, and folding techniques ---

        # List of ZNE methods
        zne_methods_list = ["linear", "richardson"]

        # List of scale factor sets to test
        scale_factors_list = [
            [1, 1.1, 1.2],
            [1, 3, 5],
            [1, 2, 3, 4, 5],
            [1, 1.1, 1.2, 1.3, 1.4]
        ]

        # List of folding techniques
        fold_methods_list = [zne.scaling.fold_gates_at_random]

        # Dictionary to store the averaged errors
        zne_avg_errors = {}

        # Iterate over each ZNE method, scale factors set, and fold method
        for zne_m in zne_methods_list:
            for scale_factors in scale_factors_list:
                # Skip invalid Richardson combos
                if zne_m == "richardson" and not all(isinstance(sf, int) for sf in scale_factors):
                    print(f"Skipping {zne_m} + {scale_factors} (partial folding not supported)")
                    continue

                for fold_method in fold_methods_list:
                    print(f"Running ZNE {zne_m=}, {scale_factors=}, {fold_method.__name__}")
                    zne_abs_errors = []

                    zne_shots_per_circ = (shots_per_check * num_checks_to_fit) // len(scale_factors)
                    print(f"  shots per circuit: {zne_shots_per_circ}")

                    # Run the circuits for this combination
                    for i, circ in enumerate(cliff_circs):
                        zne_exp = mitigate_zne(
                            circ,
                            backend,
                            pauli_string,
                            shots=zne_shots_per_circ,
                            method=zne_m,
                            scale_factors=scale_factors,
                            fold_method=fold_method,
                        )
                        print(f"  ZNE exp for circ #{i+1}: {zne_exp:.4f} (ideal {ideal_expectations[i]:.4f})")
                        zne_abs_errors.append(np.abs(ideal_expectations[i] - zne_exp))

                    avg_error = np.mean(zne_abs_errors)
                    key = f"ZNE_{zne_m}_{'_'.join(str(s) for s in scale_factors)}_{fold_method.__name__}"
                    zne_avg_errors[key] = avg_error
                    print(f"  → avg error {key}: {avg_error:.5f}\n")

        print("Final ZNE results:", zne_avg_errors)

        # --- PCE SWEEP ---

        extrapolation_methods = ["linear", "exponential"]
        pce_avg_errors       = {}

        for ext_method in extrapolation_methods:
            pce_abs_errors = []
            print(f"\nProcessing PCE {ext_method=}\n")

            for i, cliff_circ in enumerate(cliff_circs):
                print(f"  Circuit {i+1}/{num_circs}")
                expectation_values = []

                # run each depth of check
                for j in range(num_checks_to_fit):
                    pcs_circ = pcs_circs[i][j]
                    signs    = signs_list[i][j]
                    ev = ibmq_executor_pcs(
                        pcs_circ,
                        backend=backend,
                        pauli_string=pauli_string,
                        num_qubits=num_qubits,
                        signs=signs
                    )
                    expectation_values.append(ev)

                print("  raw values:", expectation_values)

                # extrapolate back to zero checks
                extrapolated_values, poly = extrapolate_checks(
                    num_checks_to_fit,
                    list(range(1, num_checks_to_fit+1)),
                    expectation_values,
                    method=ext_method
                )
                pce_exp = extrapolated_values[-1]
                print(f"  PCE exp: {pce_exp:.4f} (ideal {ideal_expectations[i]:.4f})")

                pce_abs_errors.append(np.abs(ideal_expectations[i] - pce_exp))
                print()

            avg = np.mean(pce_abs_errors)
            key = f"PCE_{ext_method}"
            pce_avg_errors[key] = avg
            print(f"Average absolute error for {key}: {avg:.5f}\n")

        print("Final PCE results:", pce_avg_errors)

        ## --- AGGREGATE & SAVE CSV ---
        circ_folder = "rand_cliffs"
        file_name   = f"avg_errors_n={num_qubits}_d={circuit_depth}_num_circs={num_circs}.csv"
        save_avg_errors(circ_folder, file_name, avg_errors, overwrite=True)

        # --- AUTOMATIC PLOTTING ---
        data_dir    = circ_folder
        num_samples = 10_000                   # match your shots or sample size
        depth       = circuit_depth

        # Load the CSV you just wrote
        data = load_avg_errors(
            data_dir,
            num_circs   = num_circs,
            num_samples= num_samples,
            depth      = depth
        )
        print(f"Loaded data for n={num_qubits}, d={depth}:", data)

        # Build a filename and save the plot
        plot_save_path = os.path.join(
            data_dir,
            f"error_plot_n={num_qubits}_d={depth}.png"
        )
        plot_avg_errors_by_qubit(
            data,
            num_samples = num_samples,
            num_circs   = num_circs,
            depth       = depth,
            save_path   = plot_save_path
        )
        print(f"✓ Plot saved to {plot_save_path}\n")
# (Optional) Turn all_results into a DataFrame for plotting/saving outside


>>> Circuit ops (ISA): OrderedDict({'rz': 28, 'cx': 15, 'x': 7, 'sx': 3})
>>> Circuit ops (ISA): OrderedDict({'rz': 32, 'cx': 15, 'x': 7, 'sx': 3})
>>> Circuit ops (ISA): OrderedDict({'rz': 32, 'cx': 17, 'x': 11, 'sx': 3})
  ZNE exp for circ #7: -0.9195 (ideal -1.0000+0.0000j)
>>> Circuit ops (ISA): OrderedDict({'cx': 21, 'rz': 20, 'x': 4, 'sx': 4})
